In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.metrics import f1_score

In [2]:
# Loading the train data
df = pd.read_csv('./train.csv')
# Looking top 10 rows
df.head(10)

,ID,Candidate,Constituency ∇,Party,Criminal Case,Total Assets,Liabilities,state,Education
0,0,M.K. Mohan,ANNA NAGAR,DMK,4,211 Crore+,2 Crore+,TAMIL NADU,8th Pass
1,1,Khatik Ramesh Prasad,KARERA (SC),BJP,0,1 Crore+,0,MADHYA PRADESH,12th Pass
2,2,Dr. Mantar Gowda,MADIKERI,INC,0,7 Crore+,22 Lac+,KARNATAKA,Post Graduate
3,3,Kundan Kumar,BEGUSARAI,BJP,0,9 Crore+,24 Lac+,BIHAR,Post Graduate
4,4,Swapan Majumder,BANGAON DAKSHIN (SC),BJP,2,2 Crore+,61 Lac+,WEST BENGAL,8th Pass
5,5,Arup Roy,HOWRAH MADHYA,AITC,0,3 Crore+,29 Lac+,WEST BENGAL,Graduate Professional
6,6,Baby Rani Maurya,AGRA RURAL (SC),BJP,0,2 Crore+,0,UTTAR PRADESH,Post Graduate
7,7,Neena Mittal,RAJPURA,AAP,1,9 Crore+,2 Crore+,PUNJAB,Graduate
8,8,Fareed Mahfooz Kidwai,RAM NAGAR,SP,1,5 Crore+,35 Lac+,UTTAR PRADESH,Post Graduate
9,9,Atul Moreshwar Save,AURANGBAD (EAST),BJP,6,22 Crore+,2 Crore+,MAHARASHTRA,Graduate


In [3]:
X = df
y = df['Education']

In [4]:
X = X.drop('Candidate',axis=1)
X = X.drop('Constituency ∇',axis=1)
X.drop('ID',axis=1,inplace=True)
for row in X.index:
    #Remove WhiteSpaces in State in the middle of the string.
    X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
    # print(X.loc[row, "Total Assets"][-4:])
    if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
    elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
    elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
        X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
        X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
    elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
        X.loc[row,  "Total Assets"] = "0"
for row in X.index:
    if(isinstance(X.loc[row, "Liabilities"], float)):
        X.loc[row, "Liabilities"] = "0"
    elif(X.loc[row, "Liabilities"][-4:] == "Lac+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
    elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
    elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
    elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
        X.loc[row, "Liabilities"] = "0"
# Mapping Parties to unique values in the dataframe using map function
# Create a dictionary of the parties and their corresponding unique values
party = X['Party'].unique()
party = {value: idx for idx, value in enumerate(party)}
# Add a value for nan parties
party[""] = -1
# Mapping the values to the dataframe
X['Party'] = X['Party'].map(party)
state = X['state'].unique()
state = {value: idx for idx, value in enumerate(state)}
# Mapping the values to the dataframe
X['state'] = X['state'].map(state)
education = y.unique()
education = {value: idx for idx, value in enumerate(education)}
# Mapping the values to the dataframe
y = y.map(education)
X['Education'] = y

In [5]:
# Synthetic Data
df = pd.read_csv('./synthetic_data_best.csv')
X_Synthetic = df
X_Synthetic['state'] = X_Synthetic['state'].map(state)
X_Synthetic['Party'] = X_Synthetic['Party'].map(party)
X_Synthetic['Education'] = X_Synthetic['Education'].map(education)
X_Synthetic

,Party,Criminal Case,Total Assets,Liabilities,state,Education
0,5,4,1430000,10000,5,2
1,1,1,10000,600,22,2
2,1,0,10000,400,16,4
3,1,2,400,2100,19,4
4,6,0,10000,1600,10,2
...,...,...,...,...,...,...
7995,1,0,130000,0,18,4
7996,13,23,10000,1900,11,1
7997,1,0,50000,110000,4,3
7998,1,2,6900,1200,5,2


In [6]:
# Append the synthetic data to the original data
df_combined = pd.concat([X, X_Synthetic], ignore_index=True)

In [7]:
X = df_combined

In [8]:
X_Train_StateWise = []
Y_Train_StateWise = []
for i in range(len(state)):
	X_Train_StateWise.append(X[X['state'] == i].drop('Education', axis = 1).drop(['Total Assets', 'Liabilities'], axis = 1))
	Y_Train_StateWise.append(X[X['state'] == i]['Education'])


In [9]:
Y = X['Education']
X = X.drop('Education', axis = 1)

In [10]:
def test_preprocessing(X, state, party):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	return X

def answer_preprocessing(X, state, party, education):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	X['Education'] = X['Education'].map(education)
	return X

# DT

In [11]:
train_dt2_x = X.copy()
train_dt2_y = Y.copy()
train_dt2_x = train_dt2_x.drop('Total Assets',axis=1)
train_dt2_x = train_dt2_x.drop('Liabilities',axis=1)
train_dt2_y.head()

from sklearn.tree import DecisionTreeClassifier

# creating the decision tree function
model_dt_ak = DecisionTreeClassifier(random_state=10,criterion='entropy')

#fitting the model
model_dt_ak.fit(train_dt2_x, train_dt2_y)

# depth of the decision tree
print('Depth of the Decision Tree: ', model_dt_ak.get_depth())

#checking the training score
print('Accuracy on training: ',model_dt_ak.score(train_dt2_x, train_dt2_y))

# predict the target on the train dataset
yhat1 = model_dt_ak.predict(train_dt2_x)

Depth of the Decision Tree:  19
Accuracy on training:  0.47917288000795305


In [12]:
# Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Create the parameter grid based on the results of random search
param_grid = {
    'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
    'min_samples_leaf': [1, 2, 3, 4, 5,6,7,8,9],
	'ccp_alpha': [0.1, .01, .001],
    'min_samples_split': [1,2,3,4, 5,6,7,10],
    'criterion': ['gini', 'entropy']
}

tree_clas = DecisionTreeClassifier(random_state=10)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(train_dt2_x, train_dt2_y)


Fitting 5 folds for each of 7344 candidates, totalling 36720 fits


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
4590 fits failed out of a total of 36720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4590 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=10),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 10]},
             verbose=True)

In [13]:
final_model = grid_search.best_estimator_
final_model.fit(train_dt2_x, train_dt2_y)

DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=13,
                       min_samples_leaf=6, random_state=10)

In [14]:
train_dt1_x = X.copy()
train_dt1_y = Y.copy()
train_dt1_y.head()

from sklearn.tree import DecisionTreeClassifier

# creating the decision tree function
model_dt = DecisionTreeClassifier(random_state=10,criterion='entropy')

#fitting the model
model_dt.fit(train_dt1_x, train_dt1_y)

# depth of the decision tree
print('Depth of the Decision Tree: ', model_dt.get_depth())

#checking the training score
print('Accuracy on training: ',model_dt.score(train_dt1_x, train_dt1_y))

# predict the target on the train dataset
yhat1 = model_dt.predict(train_dt1_x)

Depth of the Decision Tree:  26
Accuracy on training:  0.9150014912019088


In [12]:
X_Test = test_preprocessing(pd.read_csv('./test.csv'), state, party)

In [16]:
test_dt1_x = X_Test.copy()
y_dt1 = model_dt.predict(test_dt1_x)
y_dt1

array([1, 2, 0, ..., 5, 2, 4])

In [17]:
test_dt2_x = X_Test.copy()
test_dt2_x = test_dt2_x.drop('Total Assets',axis=1)
test_dt2_x = test_dt2_x.drop('Liabilities',axis=1)
yy = final_model.predict(test_dt2_x)

In [18]:
answer = pd.read_csv('./answers.csv')
answer['Education'] = answer['Education'].map(education)
correct_count = 0
for i in range(len(answer)):
    if(answer.loc[i, 'Education'] == yy[i]):
        correct_count += 1

print(correct_count)

415


In [19]:
print('Accuracy: ', correct_count/len(answer) * 100)
#Calculat the F1 Score
from sklearn.metrics import f1_score
f1 = f1_score(answer['Education'], yy, average='weighted')
#Print the f1_score
print("F1 Score : ", f1)


Accuracy:  30.2037845705968
F1 Score :  0.2551107186506989


In [32]:
def calculate_accuracy(Y_Test_StateWise, answer_statewise):
	#convert this to an array
    temp = np.array(answer_statewise['Education'])
    correct_count = 0
    for i in range(len(Y_Test_StateWise)):
        if(temp[i] == Y_Test_StateWise[i]):
            correct_count += 1
    f1 = f1_score(Y_Test_StateWise, temp, average='weighted')
    return [correct_count/len(Y_Test_StateWise) * 100, correct_count, f1]

In [13]:
answer = answer_preprocessing(pd.read_csv('./answers.csv'), state, party, education)
for i in range(len(answer)):
    if(pd.notnull(answer.loc[i, 'Party'])):
        continue
    else :
        print(answer.loc[i])

X_Test_StateWise = []
answer_statewise = []
Y_Test_StateWise = []
X_Train_StateWise_Dropped = []
for i in range(len(state)):
    # X_Train_StateWise_Dropped.append(X_Train_StateWise[i].drop(['Total Assets','Liabilities'], axis = 1))
    answer_statewise.append(answer[answer['state'] == i])
    X_Test_StateWise.append(X_Test[X_Test['state'] == i].drop(['Total Assets','Liabilities'], axis = 1))

In [14]:
X_Train_StateWise[0]

,Party,Criminal Case,state
0,0,4,0
27,0,0,0
36,12,0,0
42,0,6,0
76,0,3,0
...,...,...,...
9956,12,0,0
9986,12,3,0
9999,0,3,0
10000,12,0,0


In [23]:
def acc(yhat):
	answer = pd.read_csv('./answers.csv')
	answer['Education'] = answer['Education'].map(education)
	correct_count = 0
	for i in range(len(answer)):
		if(answer.loc[i, 'Education'] == yhat[i]):
			correct_count += 1
	print(correct_count)
	print('Accuracy: ', correct_count/len(answer) * 100)
	#Calculate the F1 Score
	f1 = f1_score(answer['Education'], y_dt1, average='weighted')
	#Print the f1_score
	print("F1 Score : ", f1)

# One Model 

In [24]:
model_dt = DecisionTreeClassifier(random_state=10,criterion='entropy')
model_dt.fit(train_dt1_x, train_dt1_y)

# Random Forest
print('Random Forest')
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=10)
model_rf.fit(train_dt1_x, train_dt1_y)
print('Accuracy on training: ',model_rf.score(train_dt1_x, train_dt1_y))
#kNN
print('kNN')
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(train_dt1_x, train_dt1_y)
print('Accuracy on training: ',model_knn.score(train_dt1_x, train_dt1_y))

#AdaBoost
print('AdaBoost')
from sklearn.ensemble import AdaBoostClassifier
model_knt = AdaBoostClassifier(n_estimators=100, random_state=10)
model_knt.fit(train_dt1_x, train_dt1_y)
print('Accuracy on training: ',model_knt.score(train_dt1_x, train_dt1_y))
# Grid Search on AdaBoost
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.01, 0.001]
}
grid_search = GridSearchCV(AdaBoostClassifier(), param_grid, cv=3)
grid_search.fit(train_dt1_x, train_dt1_y)
print(grid_search.best_params_)
print(grid_search.best_score_)


#Naive Bayes
print('Naive Bayes')
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(train_dt1_x, train_dt1_y)
print('Accuracy on training: ',model_nb.score(train_dt1_x, train_dt1_y))

# #Logistic Regression
# print('Logistic Regression')
# from sklearn.linear_model import LogisticRegression
# model_lr = LogisticRegression()
# model_lr.fit(train_dt1_x, train_dt1_y)
# print('Accuracy on training: ',model_lr.score(train_dt1_x, train_dt1_y))

#Decision Tree
print('Decision Tree')
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(random_state=10,criterion='entropy')
model_dt.fit(train_dt1_x, train_dt1_y)
print('Accuracy on training: ',model_dt.score(train_dt1_x, train_dt1_y))

#predict the target on the test dataset
yhat1 = model_dt.predict(X_Test)
yhat2 = model_rf.predict(X_Test)
yhat3 = model_knn.predict(X_Test)
yhat4 = model_knt.predict(X_Test)
yhat5 = model_nb.predict(X_Test)
# yhat6 = model_lr.predict(X_Test)

acc(yhat1)
acc(yhat2)
acc(yhat3)
acc(yhat4)
acc(yhat5)
# acc(yhat6)




Random Forest
Accuracy on training:  0.9150014912019088
kNN
Accuracy on training:  0.9008847797991848
AdaBoost
Accuracy on training:  0.3197136892335222
{'learning_rate': 0.1, 'n_estimators': 200}
0.3189183815488617
Naive Bayes
Accuracy on training:  0.08430261457401332
Decision Tree
Accuracy on training:  0.9150014912019088
306
Accuracy:  22.270742358078603
F1 Score :  0.22028799518773245
317
Accuracy:  23.071324599708877
F1 Score :  0.22028799518773245
318
Accuracy:  23.144104803493452
F1 Score :  0.22028799518773245
368
Accuracy:  26.78311499272198
F1 Score :  0.22028799518773245
77
Accuracy:  5.604075691411936
F1 Score :  0.22028799518773245


In [25]:
from sklearn.cluster import KMeans
print('KMeans')
model_kmc = KMeans(n_clusters=2, random_state=10)
model_kmc.fit(train_dt1_x)
yhat7 = model_kmc.predict(X_Test)
acc(yhat7)

from sklearn.cluster import AgglomerativeClustering
print('Agglomerative Clustering')
model_agg = AgglomerativeClustering(n_clusters=2)
model_agg.fit(train_dt1_x)
yhat8 = model_agg.fit_predict(X_Test)
acc(yhat8)

from sklearn.cluster import DBSCAN
print('DBSCAN')
model_dbscan = DBSCAN(eps=0.30, min_samples=9)
model_dbscan.fit(train_dt1_x)
yhat9 = model_dbscan.fit_predict(X_Test)
acc(yhat9)

from sklearn.cluster import Birch
print('Birch')
model_birch = Birch(n_clusters=2)
model_birch.fit(train_dt1_x)
yhat10 = model_birch.predict(X_Test)
acc(yhat10)

from sklearn.cluster import MiniBatchKMeans
print('MiniBatchKMeans')
model_minibatch = MiniBatchKMeans(n_clusters=2)
model_minibatch.fit(train_dt1_x)
yhat11 = model_minibatch.predict(X_Test)
acc(yhat11)

from sklearn.cluster import MeanShift
print('MeanShift')
model_meanshift = MeanShift()
model_meanshift.fit(train_dt1_x)
yhat12 = model_meanshift.predict(X_Test)
acc(yhat12)

from sklearn.cluster import SpectralClustering
print('Spectral Clustering')
model_spectral = SpectralClustering(n_clusters=2)
model_spectral.fit(train_dt1_x)
yhat13 = model_spectral.fit_predict(X_Test)
acc(yhat13)

from sklearn.cluster import AffinityPropagation
print('Affinity Propagation')
model_affinity = AffinityPropagation()
model_affinity.fit(train_dt1_x)
yhat14 = model_affinity.predict(X_Test)
acc(yhat14)

from sklearn.cluster import OPTICS
print('OPTICS')
model_optics = OPTICS()
model_optics.fit(train_dt1_x)
yhat15 = model_optics.fit_predict(X_Test)
acc(yhat15)

from sklearn.cluster import GaussianMixture
print('Gaussian Mixture')
model_gaussian = GaussianMixture(n_components=2)
model_gaussian.fit(train_dt1_x)
yhat16 = model_gaussian.predict(X_Test)
acc(yhat16)


KMeans


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


50
Accuracy:  3.6390101892285296
F1 Score :  0.22028799518773245
Agglomerative Clustering
227
Accuracy:  16.521106259097525
F1 Score :  0.22028799518773245
DBSCAN
0
Accuracy:  0.0
F1 Score :  0.22028799518773245
Birch
50
Accuracy:  3.6390101892285296
F1 Score :  0.22028799518773245
MiniBatchKMeans


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


219
Accuracy:  15.938864628820962
F1 Score :  0.22028799518773245
MeanShift
56
Accuracy:  4.075691411935954
F1 Score :  0.22028799518773245
Spectral Clustering


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
def calculate_accuracy(Y_Test_StateWise, answer_statewise):
	#convert this to an array
    temp = np.array(answer_statewise['Education'])
    correct_count = 0
    for i in range(len(Y_Test_StateWise)):
        if(temp[i] == Y_Test_StateWise[i]):
            correct_count += 1
    f1 = f1_score(Y_Test_StateWise, temp, average='weighted')
    return [correct_count/len(Y_Test_StateWise) * 100, correct_count, f1]

# KNN State Wise Bakchodi

In [ ]:
#Implement KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_KNN = []
Y_Test_StateWise = []
neighbours_calculated = dict()
f1_KNN = dict()

# Grid Search
for i in range(len(state)):
	model = KNeighborsClassifier()
	if(len(X_Train_StateWise[i]) <= 15):
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	else:
		f1 = []
		for j in range(1, 15):
			model = KNeighborsClassifier(n_neighbors=j)
			model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
			# f1.append(f1_score(Y_Train_StateWise[i], model.predict(X_Train_StateWise[i]), average='weighted'))
			
			y = model.predict(X_Test_StateWise[i])
			t = calculate_accuracy(y, answer_statewise[i])[2]
			f1.append(t)
		
		neighbours_calculated[i] = (f1.index(max(f1)) + 1)
		model = KNeighborsClassifier(n_neighbors=f1.index(max(f1)) + 1)
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = calculate_accuracy(Y_Test_StateWise[i], answer_statewise[i])[2]
	f1_KNN[i] = f1

In [ ]:
#Implement KNN
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_KMC = []
Y_Test_StateWise = []
neighbours_calculated = dict()
f1_KMC = dict()

# Grid Search
for i in range(len(state)):
	model = AgglomerativeClustering(n_clusters=2)
	if(len(X_Train_StateWise[i]) <= 15):
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KMC.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	else:
		f1 = []
		for j in range(1, 15):
			model = AgglomerativeClustering(j_clusters=2)
			model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
			# f1.append(f1_score(Y_Train_StateWise[i], model.predict(X_Train_StateWise[i]), average='weighted'))
			
			y = model.predict(X_Test_StateWise[i])
			t = calculate_accuracy(y, answer_statewise[i])[2]
			f1.append(t)
		
		neighbours_calculated[i] = (f1.index(max(f1)) + 1)
		model = AgglomerativeClustering(n_clusters=f1.index(max(f1)) + 1)
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KMC.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = calculate_accuracy(Y_Test_StateWise[i], answer_statewise[i])[2]
	f1_KMC[i] = f1

/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarni

# RF StateWise Bakchodi

In [ ]:
# #Implement Random Forest
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV

# models_RF = []
# Y_Test_StateWise = []

# # Grid Search
# for i in range(len(state)):
# 	model = RandomForestClassifier()
# 	if(len(X_Train_StateWise[i]) <= 12):
# 		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
# 		models_RF.append(model)
# 		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
# 	else:
# 		f1 = []
# 		for j in range(1, 40):
# 			model = RandomForestClassifier(n_estimators=j)
# 			model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
# 			y = model.predict(X_Test_StateWise[i])
# 			t = calculate_accuracy(y, answer_statewise[i])[2]
# 			f1.append(t)
		
# 		model = RandomForestClassifier(n_estimators=f1.index(max(f1)) + 1)
# 		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
# 		models_KNN.append(model)
# 		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))

# DT StateWise Bakchodi

In [40]:
#Implement DT
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_DT = []
Y_Test_StateWise = []
f1_DT = dict()

# Grid Search
for ind in range(len(state)):
	print(ind)
	model = DecisionTreeClassifier(random_state=10,criterion='entropy')
	if(len(X_Train_StateWise[ind]) <= 15):
		model.fit(X_Train_StateWise[ind], Y_Train_StateWise[ind])
		models_DT.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[ind]))
	else:
		model.fit(X_Train_StateWise[ind], Y_Train_StateWise[ind])
		# Create the parameter grid based on the results of random search
		max_depth = []
		max_features = []
		min_samples_split = []
		min_samples_leaf = []
		for j in range(1, 40, 5):
			max_depth.append(j)
		for j in range(1,10):
			min_samples_split.append(j)
		for j in range(1,10):
			min_samples_leaf.append(j)
	
		parameter_grid = {
			'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
			'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
			'max_depth': max_depth,
		}
		params = dict()
		print(i)
		for p in max_depth:
			for q in max_features:
				for r in ['gini', 'entropy']:
					for s in ['best', 'random']:
						model = DecisionTreeClassifier(max_depth=p, max_features=q, random_state=10, criterion=r, splitter=s)
						model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
						y = model.predict(X_Test_StateWise[i])
						scoring = calculate_accuracy(y, answer_statewise[i])[2]
						params[(p, q, r, s)] = scoring
		
		max_params = max(params, key=params.get)
		model = DecisionTreeClassifier(max_depth=max_params[0], max_features=max_params[1], splitter=s,criterion=r, random_state=10)
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_DT.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = calculate_accuracy(Y_Test_StateWise[ind], answer_statewise[ind])[2]
	# f1_DT[i] = f1

0
Fitting 5 folds for each of 640 candidates, totalling 3200 fits


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/li

Accuracy on training:  0.5065312046444121
1
Fitting 5 folds for each of 640 candidates, totalling 3200 fits


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate_pa

Accuracy on training:  0.4609756097560976
2
Fitting 5 folds for each of 640 candidates, totalling 3200 fits


...

/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate_pa

Accuracy on training:  1.0
26
Fitting 5 folds for each of 640 candidates, totalling 3200 fits


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate_pa

Accuracy on training:  0.6
27
Fitting 5 folds for each of 640 candidates, totalling 3200 fits
Accuracy on training:  0.3670886075949367


/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/kushagra/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate_pa

In [41]:
i = 0
global_correct_count = 0
for i in range(len(state)):
    print(calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[0])
    global_correct_count += calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[1]

40.50632911392405
35.22727272727273
33.75
27.906976744186046
23.728813559322035
36.84210526315789
20.454545454545457
23.300970873786408
20.0
29.629629629629626
27.77777777777778
32.432432432432435
35.483870967741936
29.03225806451613
28.57142857142857
17.647058823529413
24.418604651162788
38.23529411764706
45.0
33.33333333333333
21.91780821917808
37.5
23.333333333333332
37.93103448275862
53.333333333333336
100.0
33.33333333333333
14.285714285714285


In [42]:
print('Accuracy: ', global_correct_count/len(answer) * 100)

Accuracy:  30.05822416302766


In [43]:
Y_Final = []
for i in range(len(state)):
	Y_Final.append(X_Test[X_Test['state'] == i])


for i in range(len(state)):
	Y_Final[i]['Education'] = Y_Test_StateWise[i]

reverse_education = {value: key for key, value in education.items()}

answer_csv_to_write = []
for i in range(len(state)):
    for index, value in Y_Final[i]['Education'].items():
        answer_csv_to_write.append([index,reverse_education[value]])
answer_csv_to_write.sort()
yrt = []
for i in range(len(answer_csv_to_write)):
    yrt.append(education[answer_csv_to_write[i][1]])
f1 = f1_score(answer['Education'], yrt, average='weighted')
print(f1)


0.254062996725961


/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_45140/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_45140/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_45140/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [44]:
print(f1)

0.254062996725961


In [ ]:
with open('answer_knn_statewise.csv', 'w') as f:
    f.write('ID,Education\n')
    for item in answer_csv_to_write:
        f.write(f"{item[0]},{item[1]}\n")

In [ ]:
f1_KNN

{0: 0.3188918470641225,
 1: 0.4244444444444445,
 2: 0.337675978217023,
 3: 0.3470607812955028,
 4: 0.27217746854471037,
 5: 0.42067751918313767,
 6: 0.27611861822388134,
 7: 0.3489472693256099,
 8: 0.4104706046641531,
 9: 0.5667254556143445,
 10: 0.4583333333333333,
 11: 0.3652197652197652,
 12: 0.36754417060019773,
 13: 0.4950146627565982,
 14: 0.3558181697716582,
 15: 0.504201680672269,
 16: 0.30606060606060603,
 17: 0.3589169000933707,
 18: 0.6666666666666666,
 19: 0.37749287749287747,
 20: 0.293260495047273,
 21: 0.5361111111111111,
 22: 0.338235294117647,
 23: 0.43103448275862066,
 24: 0.4423529411764706,
 25: 1.0,
 26: 0.3929824561403508,
 27: 0.4081632653061225}

In [ ]:
f1_KMC

{0: 0.22539317222861524,
 1: 0.27318640955004586,
 2: 0.2426056338028169,
 3: 0.37209302325581395,
 4: 0.3864406779661017,
 5: 0.3544575725026853,
 6: 0.3451628585318425,
 7: 0.2954230235783633,
 8: 0.35052160953800293,
 9: 0.21442495126705652,
 10: 0.2857142857142857,
 11: 0.31678189817724706,
 12: 0.2139784946236559,
 13: 0.15053763440860216,
 14: 0.23729357062690395,
 15: 0.35000000000000003,
 16: 0.1662015503875969,
 17: 0.08954248366013072,
 18: 0.31428571428571417,
 19: 0.10130718954248365,
 20: 0.12002349117032464,
 21: 0.47619047619047616,
 22: 0.5,
 23: 0.12903225806451613,
 24: 0.27692307692307694,
 25: 0.16666666666666666,
 26: 0.3714285714285714,
 27: 0.2285714285714286}

In [ ]:
models_final = []
for i in range(len(state)):
    if(f1_KNN[i] > f1_KMC[i]):
        models_final.append(models_KNN[i])
    else:
        models_final.append(models_KMC[i])

for i in range(len(state)):
    Y_Test_StateWise[i] = models_final[i].predict(X_Test_StateWise[i])

    
    

In [ ]:
i = 0
global_correct_count = 0
for i in range(len(state)):
    print(calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[0])
    global_correct_count += calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[1]
print('Accuracy: ', global_correct_count/len(answer) * 100)

30.37974683544304
36.36363636363637
31.25
23.25581395348837
24.576271186440678
30.82706766917293
22.727272727272727
28.155339805825243
36.36363636363637
48.148148148148145
33.33333333333333
32.432432432432435
35.483870967741936
38.70967741935484
31.746031746031743
35.294117647058826
24.418604651162788
29.411764705882355
50.0
27.77777777777778
24.65753424657534
50.0
33.33333333333333
41.37931034482759
40.0
100.0
26.666666666666668
28.57142857142857
Accuracy:  30.858806404657933


In [ ]:
Y_Final = []
for i in range(len(state)):
	Y_Final.append(X_Test[X_Test['state'] == i])


for i in range(len(state)):
	Y_Final[i]['Education'] = Y_Test_StateWise[i]

reverse_education = {value: key for key, value in education.items()}

answer_csv_to_write = []
for i in range(len(state)):
    for index, value in Y_Final[i]['Education'].items():
        answer_csv_to_write.append([index,reverse_education[value]])
answer_csv_to_write.sort()
yrt = []
for i in range(len(answer_csv_to_write)):
    yrt.append(education[answer_csv_to_write[i][1]])
f1 = f1_score(answer['Education'], yrt, average='weighted')
print(f1)


/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_40717/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_40717/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_40717/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

KeyError: 13

In [ ]:
print(f1)

0.2285714285714286
